In [3]:
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import display
import pandas as pd
import json
import numpy as np
from pandas.io.json import json_normalize

pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 999


def inf(column):
    return np.log(17608989/float(column))
    
def execute_query(sparqlQuery):
    sparql = SPARQLWrapper("http://zen.cs.rpi.edu:8890/sparql")
    sparql.setQuery(sparqlQuery)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = json_normalize(results["results"]["bindings"])
    results_df = results_df[results_df['o.type'] == "uri"]
    results_df = results_df.drop(['o.datatype','o.type', 'o.xml:lang','p.type'],axis=1)
    results_df = results_df[['p.value','o.value','countS.value']]
    results_df['countS.value'] = results_df['countS.value'].astype(int)
    results_df = results_df[results_df['countS.value'] > 1]
    return results_df


def execute_count_query(obj):
    sparqlQuery = "Select (count(*) as ?countRow)  WHERE { ?s " + "?p  <" + obj + "> }"
    sparql = SPARQLWrapper("http://zen.cs.rpi.edu:8890/sparql")
    sparql.setQuery(sparqlQuery)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = json_normalize(results["results"]["bindings"])
    val = float(results_df['countRow.value'])
    return val


query = """
PREFIX  schema: <http://schema.org/>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX  umbel-rc: <http://umbel.org/umbel/rc/>
PREFIX  dbc:  <http://dbpedia.org/resource/Category:>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  yago: <http://dbpedia.org/class/yago/>
PREFIX  rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  dbo:  <http://dbpedia.org/ontology/>
PREFIX  dbp:  <http://dbpedia.org/property/>
PREFIX  rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  dcterms: <http://purl.org/dc/terms/>
PREFIX  dbpedia-wikidata: <http://wikidata.dbpedia.org/resource/>
PREFIX  dul:  <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX  foaf: <http://xmlns.com/foaf/0.1/>
PREFIX  dc:   <http://purl.org/dc/elements/1.1/>
PREFIX prov: <http://www.w3.org/ns/prov#>

SELECT  ?p ?o (COUNT(DISTINCT ?s) AS ?countS)
WHERE
  { dbr:Martin_Scorsese
              ?p  ?o .
    ?s        ?p  ?o
    FILTER ( ! strstarts(str(?p), str(dbo:wikiPageWikiLink)) )
    FILTER ( ! strstarts(str(?p), str(dbo:wikiPageID)) )
    FILTER ( ! strstarts(str(?p), str(dbo:wikiPageExternalLink)) )
    FILTER ( ! strstarts(str(?p), str(dbo:wikiPageRevisionID)) )
    FILTER ( ! strstarts(str(?p), str(dbo:wikiPageLength)) )
    FILTER ( ! strstarts(str(?p), str(owl:sameAs)) )
    FILTER ( ! strstarts(str(?p), str(dbo:viafId)) )
    FILTER ( ! strstarts(str(?p), str(dbo:wikiPageOutDegree)) )
    FILTER ( ! strstarts(str(?p), str(schema:comment)) )
    FILTER ( ! strstarts(str(?p), str(dbo:abstract)) )
    FILTER ( ! strstarts(str(?p), str(rdfs:comment)) )
    FILTER ( ! strstarts(str(?p), str(dbo:alias)) )
    FILTER ( ! strstarts(str(?p), str(rdfs:label)) )
    FILTER ( ! strstarts(str(?p), str(dbo:thumbnail)) )
    FILTER ( ! strstarts(str(?p), str(foaf:name)) )
    FILTER ( ! strstarts(str(?p), str(foaf:surname)) )
    FILTER ( ! strstarts(str(?p), str(dcterms:subject)) )
    FILTER ( ! strstarts(str(?p), str(foaf:depiction)) )
    FILTER ( ! strstarts(str(?p), str(foaf:isPrimaryTopicOf)) )
    FILTER ( ! strstarts(str(?p), str(dbp:hasPhotoCollection)) )
    FILTER ( ! strstarts(str(?p), str(dbp:wordnet_type)) )
    FILTER ( ! strstarts(str(?p), str(prov:wasDerivedFrom)) )
    FILTER(!strstarts(str(?p),str(rdf:type)))
    
    
  }
GROUP BY ?p ?o
ORDER BY ASC(?countS)
"""

countQuery = """


"""
results_df = execute_query(query)
results_df.reset_index(inplace=True)
# results_df = results_df.sort_values(by=['countS.value'],ascending=True)

results_df['INF'] = results_df['countS.value'].apply(inf)
results_df['Pop'] = results_df['o.value'].apply(execute_count_query)
results_df['Rank'] = results_df['Pop'] * results_df['INF']
results_df.sort_values(by=['Rank'],ascending=False,inplace=True)
results_df.sort_values(by=['Rank'],ascending=False,inplace=True)
display(results_df)
# results
# for result in results["results"]["bindings"]:
#     print(result["p"]["value"] + " : " + result["o"]["value"]) 
    
# review_df = pd.DataFrame(sparql,index=[0])
# review_df.head()

# from gensim.models import Word2Vec
# model = Word2Vec.load("/home/amar/Data/en_1000_no_stem/en.model")
# model.similarity('woman', 'man')


,index,p.value,o.value,countS.value,INF,Pop,Rank
11,39,http://dbpedia.org/ontology/birthPlace,http://dbpedia.org/resource/New_York,6882,7.847255,100591.0,789363.276180
12,41,http://dbpedia.org/ontology/religion,http://dbpedia.org/resource/Catholic_Church,8149,7.678270,79871.0,613271.068741
13,42,http://dbpedia.org/property/religion,http://dbpedia.org/resource/Catholic_Church,8843,7.596539,79871.0,606743.134815
5,27,http://dbpedia.org/property/almaMater,http://dbpedia.org/resource/New_York_University,358,10.803387,8937.0,96549.870351
6,28,http://dbpedia.org/ontology/almaMater,http://dbpedia.org/resource/New_York_University,417,10.650834,8937.0,95186.502086
7,29,http://dbpedia.org/ontology/stateOfOrigin,http://dbpedia.org/resource/Americans,422,10.638915,8874.0,94409.729529
9,36,http://dbpedia.org/ontology/nationality,http://dbpedia.org/resource/Americans,1484,9.381424,8874.0,83250.753422
10,37,http://dbpedia.org/property/nationality,http://dbpedia.org/resource/Americans,1514,9.361410,8874.0,83073.149094
8,31,http://dbpedia.org/ontology/birthPlace,http://dbpedia.org/resource/Queens,516,10.437813,6743.0,70382.175101
4,26,http://dbpedia.org/ontology/birthPlace,"http://dbpedia.org/resource/Flushing,_Queens",78,12.327211,1170.0,14422.837153
